In [ ]:
%cd ../..

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
# mongoexport --uri mongodb+srv://efiss:*@efiss.wj88voa.mongodb.net/efiss --collection products --type json --out output-shopee.json --query '{"url": {"$regex": "https://shopee.vn"}}'

db_file = 'data/shopee/output-shopee.json'

data = []
with open(db_file, 'r') as f:
    for line in tqdm(f):
        data.append(json.loads(line))

In [ ]:
bdf = pd.DataFrame(data)
bdf.dropna(subset=['originalImages'], inplace=True)
df = bdf
# df = bdf.iloc[:1000]
df

In [ ]:
df = df[['title', 'images', 'categories']]
df['categories'] = df['categories'].apply(lambda x: " > ".join(x))
df

In [ ]:
df['categories'].value_counts()

In [ ]:
temp = df['categories'].value_counts()
temp = temp[temp > 10]
temp

In [ ]:
temp.to_csv('data/shopee/cls.csv', index=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(temp)

... After manually filter all fashion-unrelated labels

In [ ]:
temp = pd.read_csv('data/shopee/cls.csv', index_col=0)
temp

### We have 272 class left, we will use them

How many ``images = []`` is missing?

In [ ]:
sum(df['images'].apply(lambda x: len(x)) == 0) / len(df)

In [ ]:
sum(bdf['originalImages'].apply(lambda x: len(x)) == 0) / len(df)

### Generate dataset for training

In [ ]:
bdf.shape

In [ ]:
temp.index

In [ ]:
adf = df[df['categories'].isin(temp.index)]
adf

In [ ]:
adf.shape, df.shape

Data reduce from 110434 to 107356

In [ ]:
# convert categories to class index
class_to_idx = {c: i for i, c in enumerate(temp.index)}
class_to_idx

In [ ]:
# convert categories to class index
adf['class_idx'] = adf['categories'].apply(lambda x: class_to_idx[x])
adf

In [ ]:
adf

In [ ]:
pdf = adf[['images', 'class_idx']]
pdf = pdf.explode('images')
pdf

In [ ]:
pdf['images'] = "data/shopee/product_images" + pdf['images'].str.split('product_images').str[-1]
pdf['images']

In [ ]:
pdf.set_index('images', inplace=True)

In [ ]:
pdf

In [ ]:
pdf.reset_index(drop=False, inplace=True)
pdf

In [ ]:
pdf.to_csv('data/shopee/shopee-data-for-training.csv', index=False)

In [ ]:
import os
from PIL import Image


def check_img_integrity(img_path):
    try:
        with open(img_path, 'rb') as f:
            check_chars = f.read()[-2:]
        if check_chars != b'\xff\xd9':
            print(f"Not complete image: {img_path}")
            return False

        with Image.open(img_path) as img:
            img.verify()
            image_format = img.format.lower()
            if image_format == 'jpeg' and not (img_path.endswith('.jpg') or img_path.endswith('.jpeg')):
                print(f"Wrong image extension: {img_path}")
                return False

        return True
    except Exception as e:
        print(e)
        return False

In [ ]:
test = pdf.head(1000)
test

In [45]:
mask = pdf.images.progress_apply(check_img_integrity)

  0%|          | 0/738613 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-22decembrestore/6460aae452e365505c04bca9_4_shopee_22decembrestore.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-22decembrestore/6460aaf852e365505c04bd66_4_shopee_22decembrestore.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-thedelia/6460ab0152e365505c04bdc4_6_shopee_thedelia.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-thedelia/6460ab0152e365505c04bdc4_7_shopee_thedelia.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-thedelia/6460ab0152e365505c04bdc4_8_shopee_thedelia.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-rame_vn/6460c04a48de6f0ca36bea35_7_shopee_rame_vn.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/shopee-rame_vn/6460c04a48de6f0ca36bea35_8_shopee_rame_vn.jpeg'
[Errno 2] No such file or directory: 'data/shopee/product_images/s

In [50]:
mask

0         True
1         True
2         True
3         True
4         True
          ... 
738608    True
738609    True
738610    True
738611    True
738612    True
Name: images, Length: 738613, dtype: bool

In [46]:
print(f"Corrupted images: {(~mask).sum()}")

Corrupted images: 2573


In [47]:
print(f"Num images: {len(mask)}")

Num images: 738613


In [56]:
database = pdf[mask]

In [57]:
database.to_csv('data/shopee/shopee-data-for-training-filter-corrupted.csv', index=False)

In [58]:
database

,images,class_idx
0,data/shopee/product_images/shopee-thedelia/646...,24
1,data/shopee/product_images/shopee-thedelia/646...,24
2,data/shopee/product_images/shopee-thedelia/646...,24
3,data/shopee/product_images/shopee-thedelia/646...,24
4,data/shopee/product_images/shopee-22decembrest...,0
...,...,...
738608,data/shopee/product_images/shopee-4lova/6462d7...,31
738609,data/shopee/product_images/shopee-4lova/6462d7...,31
738610,data/shopee/product_images/shopee-4lova/6462d7...,31
738611,data/shopee/product_images/shopee-4lova/6462d7...,31


In [59]:
database.to_csv('data/shopee/database.txt', sep=' ', header=False, index=False)

In [77]:
# sample 100 images from each class, if the class has less than that amount of images, we sample all images from that class
def sample(df, n=10):
    if len(df) > n:
        return df.sample(n=n)
    else:
        return df

In [78]:
test_df = database.groupby(['class_idx']).progress_apply(sample).reset_index(drop=True)

  0%|          | 0/271 [00:00<?, ?it/s]

In [79]:
test_df

,images,class_idx
0,data/shopee/product_images/shopee-somehow/6462...,0
1,data/shopee/product_images/shopee-emix.vn/6462...,0
2,data/shopee/product_images/shopee-depshop8888/...,0
3,data/shopee/product_images/shopee-rechic/64629...,0
4,data/shopee/product_images/shopee-vayemy/64624...,0
...,...,...
2705,data/shopee/product_images/shopee-thoitrangthe...,271
2706,data/shopee/product_images/shopee-usthebasic/6...,271
2707,data/shopee/product_images/shopee-thoitrangthe...,271
2708,data/shopee/product_images/shopee-hoangphucint...,271


In [80]:
test_df.to_csv('data/shopee/test.txt', sep=' ', header=False, index=False)

In [81]:
!cat data/shopee/database.txt | head

data/shopee/product_images/shopee-thedelia/6460ab0252e365505c04bdd6_0_shopee_thedelia.jpeg 24
data/shopee/product_images/shopee-thedelia/6460ab0252e365505c04bdd6_1_shopee_thedelia.jpeg 24
data/shopee/product_images/shopee-thedelia/6460ab0252e365505c04bdd6_2_shopee_thedelia.jpeg 24
data/shopee/product_images/shopee-thedelia/6460ab0252e365505c04bdd6_3_shopee_thedelia.jpeg 24
data/shopee/product_images/shopee-22decembrestore/6460aaee52e365505c04bd06_0_shopee_22decembrestore.jpeg 0
data/shopee/product_images/shopee-22decembrestore/6460aaee52e365505c04bd06_1_shopee_22decembrestore.jpeg 0
data/shopee/product_images/shopee-22decembrestore/6460aaee52e365505c04bd06_2_shopee_22decembrestore.jpeg 0
data/shopee/product_images/shopee-22decembrestore/6460aafe52e365505c04bda5_0_shopee_22decembrestore.jpeg 112
data/shopee/product_images/shopee-22decembrestore/6460aafe52e365505c04bda5_1_shopee_22decembrestore.jpeg 112
data/shopee/product_images/shopee-22decembrestore/6460aafe52e365505c04bda5_2_shopee_2

In [82]:
!cat data/shopee/test.txt | head

data/shopee/product_images/shopee-somehow/64621d16614eb205453391de_5_shopee_somehow.jpeg 0
data/shopee/product_images/shopee-emix.vn/64624b7b614eb2054534877e_4_shopee_emix_vn.jpeg 0
data/shopee/product_images/shopee-depshop8888/64624504614eb205453469a4_5_shopee_depshop8888.jpeg 0
data/shopee/product_images/shopee-rechic/6462941d614eb2054535d68b_8_shopee_rechic.jpeg 0
data/shopee/product_images/shopee-vayemy/64624e9a614eb2054534952b_1_shopee_vayemy.jpeg 0
data/shopee/product_images/shopee-lovito.vn/64626a28614eb2054535261a_2_shopee_lovito_vn.jpeg 0
data/shopee/product_images/shopee-lyzastudios/64627510614eb20545358258_1_shopee_lyzastudios.jpeg 0
data/shopee/product_images/shopee-ivymoda/646210f4614eb2054532eb3b_8_shopee_ivymoda.jpeg 0
data/shopee/product_images/shopee-libeworkshop/64626ec9614eb205453564db_2_shopee_libeworkshop.jpeg 0
data/shopee/product_images/shopee-cecicela/6462305b614eb2054534460c_2_shopee_cecicela.jpeg 0
cat: write error: Broken pipe
